In [4]:
import tweepy
import json
import re

class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 20
        self.count = 1
        self.all_tweets = ""
        self.all_tweets_list = []
    
    def on_status(self, status):
        print(status.text)
        
    def on_data(self, data):
        tweet = json.loads(data)
        if not tweet['retweeted'] and 'RT @' not in tweet['text']:  # remove duplicate tweets
            if self.count == (self.num_tweets+1):
                with open('data.json', 'w') as outfile:
                    json.dump(self.all_tweets_list, outfile)
                return False
            t = dict()
            t['id'] = self.count
            # remove all emoticons and any other special symbols.
            emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"
                           u"\U0001F1E0-\U0001F1FF"
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
            tweet['text'] = emoji_pattern.sub(r'', tweet['text'])
            t['text'] = tweet['text']
            self.count += 1
            self.all_tweets += (json.dumps(t)+"\n")
            self.all_tweets_list.append(t)  # save 
        return True
    
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
    def get_all_tweets(self):
        return self.all_tweets
        
ACCESS_TOKEN = '1040311982015037440-1TSqPfFZmXn1fUS1qplchjVbWBMANn'
ACCESS_SECRET = 'K0efO3na1D9Mw1Du7rkROzgOW8zDTLTG6AcVTuLqMzjQq'
CONSUMER_KEY = 'j4kLoamWfoPbw72VLoQwjmzeF'
CONSUMER_SECRET = 'UwEGlKxK2xX4I59FXhwUmc1A4wPoW2ZceTY1ZI2iTdEGDOaOjY'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.secure = True
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=MyStreamListener())
myStream.filter(track=['lol'])


In [3]:
!pip install tweepy

    100% |################################| 61kB 1.7MB/s eta 0:00:01
    100% |################################| 286kB 2.0MB/s eta 0:00:01
    100% |################################| 122kB 3.1MB/s eta 0:00:01
    100% |################################| 153kB 2.9MB/s eta 0:00:01
    100% |################################| 143kB 3.0MB/s eta 0:00:01
    100% |################################| 61kB 1.7MB/s eta 0:00:01
    100% |################################| 122kB 2.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for PySocks ... done
  Stored in directory: /home/spark/shared/.cache/pip/wheels/22/5c/b5/12e0dfdfa85bea67b23628b6425fae715c687e947a45ee3df9
Successfully built PySocks


In [5]:
import pyspark 
from pyspark.sql import SQLContext

spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()
sc = SparkContext.getOrCreate()
sqlc = SQLContext(sc)
df = sqlc.read.json("data.json",multiLine=True)
df.show()

+---+--------------------+
| id|                text|
+---+--------------------+
|  1|I’m getting me a ...|
|  2|       Chill out lol|
|  3|nunca achei que e...|
|  4|Narrator:  They d...|
|  5|@ayeetrey_ lol yo...|
|  6|      Trae Young lol|
|  7|I dont know about...|
|  8|Talk to em ! Pipe...|
|  9|lol i guess not w...|
| 10|@Lift_andLove Lol...|
| 11|anyways atiku lat...|
| 12|So Spotify follow...|
| 13|@JChanDaLand Lol ...|
| 14|a vontade que eu ...|
| 15|@Lulubookseller @...|
| 16|Lol and there you...|
| 17|Now this is a lit...|
| 18|Ya lo ví todo. Pa...|
| 19|@alex_robinson04 ...|
| 20|Hell yeah @Succes...|
+---+--------------------+



In [6]:
# The code was removed by Watson Studio for sharing.

+-----+--------------------+
|label|                text|
+-----+--------------------+
|    4|Reading my kindle...|
|    4|Ok, first assesme...|
|    4|@kenburbary You'l...|
|    4|@mikefish  Fair e...|
|    4|@richardebaker no...|
|    0|Fuck this economy...|
|    4|Jquery is my new ...|
|    4|       Loves twitter|
|    4|how can you not l...|
|    2|Check this video ...|
|    0|@Karoli I firmly ...|
|    4|House Corresponde...|
|    4|Watchin Espn..Jus...|
|    0|dear nike, stop w...|
|    4|#lebron best athl...|
|    0|I was talking to ...|
|    4|i love lebron. ht...|
|    0|@ludajuice Lebron...|
|    4|@Pmillzz lebron I...|
|    4|@sketchbug Lebron...|
+-----+--------------------+
only showing top 20 rows



In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.sql.functions import lit
from pyspark.sql import types

df_data_2 = df_data_2.withColumn("label", df_data_2.label.cast("Int"))

(train, test) = df_data_2.randomSplit([0.9, 0.1], seed = 1000)

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=20,regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model= pipeline.fit(train)

# Make predictions on test documents and print columns of interest.
predictions = model.transform(test)
#predictions.show()

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("F1: %g" % (evaluator.evaluate(predictions)))

#save the model, overwrite if the model already exist
#model.save("logreg.model")
model.write().overwrite().save("logreg.model")


F1: 0.674556


In [8]:
from pyspark.ml import PipelineModel
pipelineModel = PipelineModel.load("logreg.model")
prediction = pipelineModel.transform(df)
prediction.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|                text|               words|            features|       rawPrediction|         probability|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  1|I’m getting me a ...|[i’m, getting, me...|(262144,[9639,223...|[1.66040780300058...|[0.10698119672902...|       4.0|
|  2|       Chill out lol|   [chill, out, lol]|(262144,[21007,31...|[2.11125231084676...|[0.23909885459871...|       4.0|
|  3|nunca achei que e...|[nunca, achei, qu...|(262144,[3934,507...|[1.99762596666603...|[0.21116663227740...|       2.0|
|  4|Narrator:  They d...|[narrator:, , the...|(262144,[60108,61...|[3.72277660856545...|[0.74898544122468...|       0.0|
|  5|@ayeetrey_ lol yo...|[@ayeetrey_, lol,...|(262144,[1536,951...|[8.07048891535554...|[0.99941090139007...|       0.0|
|  6|      Trae Young lo

In [9]:
#show positive tweets
prediction.filter(prediction['prediction'] == 4).select("id","text","prediction").show(10)

#show natural tweets
prediction.filter(prediction['prediction'] == 2).select("id","text","prediction").show(10)

#show negative tweets
prediction.filter(prediction['prediction'] == 0).select("id","text","prediction",).show(10)

+---+--------------------+----------+
| id|                text|prediction|
+---+--------------------+----------+
|  1|I’m getting me a ...|       4.0|
|  2|       Chill out lol|       4.0|
|  6|      Trae Young lol|       4.0|
|  9|lol i guess not w...|       4.0|
| 10|@Lift_andLove Lol...|       4.0|
| 12|So Spotify follow...|       4.0|
| 14|a vontade que eu ...|       4.0|
| 15|@Lulubookseller @...|       4.0|
| 16|Lol and there you...|       4.0|
+---+--------------------+----------+

+---+--------------------+----------+
| id|                text|prediction|
+---+--------------------+----------+
|  3|nunca achei que e...|       2.0|
| 13|@JChanDaLand Lol ...|       2.0|
+---+--------------------+----------+

+---+--------------------+----------+
| id|                text|prediction|
+---+--------------------+----------+
|  4|Narrator:  They d...|       0.0|
|  5|@ayeetrey_ lol yo...|       0.0|
|  7|I dont know about...|       0.0|
|  8|Talk to em ! Pipe...|       0.0|
| 11|anywa